In [1]:
import pandas as pd
import numpy as np

In [2]:
%ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is 220E-4420

 Directory of c:\Users\lenha\Repos\fantasy-baseball-draft\CBS_fantasy_baseball

03/27/2021  03:09 PM    <DIR>          .
03/27/2021  03:09 PM    <DIR>          ..
03/05/2021  07:30 AM                 0 __init__.py
03/05/2021  07:30 AM               290 assistant.py
03/05/2021  07:30 AM            21,785 BaseFantasyClasses.py
03/27/2021  03:09 PM            33,141 development_notebook.ipynb
03/05/2021  07:30 AM    <DIR>          file_handlers
03/05/2021  07:30 AM             1,011 pitchers.py
03/05/2021  07:30 AM    <DIR>          player
03/05/2021  07:30 AM            13,513 stats.py
03/05/2021  07:30 AM    <DIR>          team
               6 File(s)         69,740 bytes
               5 Dir(s)  77,602,246,656 bytes free


In [3]:
df1 = pd.read_csv('../data/projections/2021/bat hitters.csv')
df2 = pd.read_csv('../data/projections/2021/bat x hitters.csv')
df3 = pd.read_csv('../data/projections/2021/steamer batters.csv')
df4 = pd.read_csv('../data/projections/2021/zips hitters.csv')

In [4]:
dfs = [df1, df2, df3, df4]
score_cats = ['R','HR', 'RBI', 'H', 'AB', 'SB']
bio_cats = ['Name', 'playerid','Team','PA']
for df in dfs:
    for c in score_cats:
        if c not in df.columns:
            print('Ahk!')
    for c in bio_cats:
        if c not in df.columns:
            print("Yeesh!")

In [5]:
def df_to_fb_stats(df):
    if df['HR'].dtype:
        pass
    df = df[bio_cats + score_cats].copy()
    for col in score_cats:
        df[col] = df[col]/df['PA']
    return df
dfs2 = []
for df in dfs:
    df = df_to_fb_stats(df)
    dfs2.append(df)
dfs2[0].head()
dfs = dfs2

In [6]:
merged = None
merged = pd.merge(dfs[0], dfs[1], 'outer', left_on='playerid', right_on='playerid', suffixes=('_bat', '_xbat'))
merged = pd.merge(merged, dfs[2], 'outer', left_on='playerid', right_on='playerid')
merged = pd.merge(merged, dfs[3], 'outer', left_on='playerid', right_on='playerid', suffixes=('_stmr', '_zips'))

In [7]:
merged.isna()
#def agg_score_cols(df):
cols = merged.columns.str.startswith('Team')
cols = merged.columns[cols]
def set_team(series):
    if series.isna().all():
        return np.nan
    else:
        return series.dropna().iloc[0]
merged['Team'] = merged[cols].apply(set_team, axis = 1)
cols = merged.columns.str.startswith('Name')
cols = merged.columns[cols]
merged['Name'] = merged[cols].apply(lambda x: x.dropna().iloc[0], axis=1)
merged.iloc[0].dropna()
for stat in score_cats:
    cols = merged.columns.str.startswith(stat)
    merged[stat] = merged[merged.columns[cols]].apply(np.mean, axis = 1)


In [8]:
merged = merged[bio_cats[:-1] + score_cats]

In [125]:
merged.head()
merged.to_csv('../data/projections/2021/merged_hitter_scoring.csv')

In [10]:
pt = pd.read_csv('../data/projections/2021/fg_depth hitters.csv')
merged = pd.merge(merged, pt[['PA','playerid']], how='outer', left_on='playerid', right_on='playerid')
merged.head()

,Name,playerid,Team,R,HR,RBI,H,AB,SB,PA
0,Mike Trout,10155,LAA,0.168681,0.063104,0.167421,0.135713,0.795689,0.016909,679.0
1,Juan Soto,20123,WSN,0.172518,0.055979,0.177381,0.140262,0.819014,0.018970,665.0
2,Freddie Freeman,5361,ATL,0.155080,0.045781,0.156940,0.137330,0.849179,0.009746,679.0
3,Christian Yelich,11477,MIL,0.149860,0.051534,0.144707,0.133858,0.846531,0.027127,672.0
4,Bryce Harper,11579,PHI,0.157180,0.054264,0.159416,0.125204,0.814496,0.020378,665.0


In [16]:
#pd.read_csv('../data/projections/2021/cbs hitters.csv', skiprows=1, skipfooter=1)
id_map = pd.read_csv('../data/projections/2021/id_map.txt', engine='python', sep='\t')
#pd.read_csv?
#drop = id_map.columns.where(id_map.columns.str.startswith('Unnamed')).dropna()
#id_map.drop(drop, inplace=True, axis=1)
(id_map['mlb_name'] != id_map['fg_name']).sum()

705

In [60]:
id_map[['cbs_id', 'cbs_name', 'cbs_pos', 'espn_id', 'espn_name', 'espn_pos', 'fg_id', 'fg_name', 'fg_pos']]
id_map['cbs_id'] = id_map['cbs_id'].dropna().astype(int)

In [61]:
id_map.head()

,mlb_id,mlb_name,mlb_pos,mlb_team,mlb_team_long,bats,throws,birth_date,bp_id,bref_id,...,retro_name,debut,yahoo_id,yahoo_name,ottoneu_id,ottoneu_name,ottoneu_pos,rotowire_id,rotowire_name,rotowire_pos
0,592091,A.J. Achter,P,LAA,Los Angeles Angels,R,R,8/27/1988,66919.0,achteaj01,...,A.J. Achter,9/3/2014,9824.0,A.J. Achter,18093.0,A.J. Achter,RP,NaN,NaN,NaN
1,595918,A.J. Cole,P,TOR,Toronto Blue Jays,R,R,1/5/1992,68086.0,coleaj01,...,A.J. Cole,4/28/2015,9638.0,A.J. Cole,14940.0,A.J. Cole,RP,11446.0,A.J. Cole,P
2,454560,A.J. Ellis,C,SD,San Diego Padres,R,R,4/9/1981,47564.0,ellisaj01,...,A.J. Ellis,9/15/2008,8373.0,A.J. Ellis,1543.0,A.J. Ellis,Util,9242.0,A.J. Ellis,C
3,456167,A.J. Griffin,P,TEX,Texas Rangers,R,R,1/28/1988,65757.0,griffaj01,...,A.J. Griffin,6/24/2012,9220.0,A.J. Griffin,13764.0,A.J. Griffin,RP,12410.0,A.J. Griffin,P
4,543362,A.J. Jimenez,C,TEX,Texas Rangers,R,R,5/1/1990,58320.0,jimenaj01,...,A.J. Jimenez,9/6/2017,NaN,NaN,17777.0,A.J. Jimenez,Util,12071.0,A.J. Jimenez,C


In [31]:
'Jack Cust'.split(' ')

['Jack', 'Cust']

In [46]:
cbs = pd.read_csv('../data/projections/2021/cbs hitters.csv', engine='python', skiprows=1, skipfooter=1)
cbs.drop('Unnamed: 18', inplace=True, axis = 1)

def cbs_player_to_name(player):
    '''convets cbs's dumb player format to a name.

    Format of player is like 
        First [Middle] Last [Suffix] Pos | Team_short
    Nice clean data, fellas.
    '''
    player = player.split('|')[0].strip()
    player = " ".join(player.split(' ')[:-1])

    return player
cbs['Name'] = cbs['Player'].apply(cbs_player_to_name)
cbs['Team'] = cbs['Player'].apply(lambda x: x.split('|')[1].strip())

cbs[cbs['Avail'] != 'W '].head()
#cbs.iloc[0]['Avail']
elig = cbs[['Avail', 'Name', 'Team', 'Eligible', 'Player']]

In [59]:
for team in id_map['mlb_team'].unique():
    if team not in elig['Team'].unique():
        print(team)
team_dict = {'CWS':'CHW', 'WSH':'WAS'}
cbs['Team'] = cbs['Team'].apply(lambda x: team_dict[x] if x in ['CWS', 'WSH'] else x)
id_map['mlb_team'] = id_map['mlb_team'].apply(lambda x: team_dict[x] if x in ['CWS', 'WSH'] else x)
cbs.merge(id_map['mlb_team', 'cbs_name', 'cbs_id'], 'left', left_on=('Name','Team'), right_on=('cbs_name','mlb_team')

WSH
CWS


In [122]:
id_map.dropna(subset=['cbs_name'])
id_map.dropna(subset=['cbs_name']).duplicated(subset=('cbs_name', 'mlb_team')).sum()
dups = cbs['Name'].duplicated(keep=False)
cbs[['Name','Team']][dups].sort_values('Name')

,Name,Team
2658,Alexander Ramirez,NYM
2612,Alexander Ramirez,LAA
1721,Anthony Garcia,LAD
2281,Anthony Garcia,NYY
188,Carlos Perez,CHW
2821,Carlos Perez,OAK
2593,Cristian Santana,LAD
2270,Cristian Santana,DET
627,Jose Martinez,MIN
2606,Jose Martinez,NYM


In [123]:
# Next....
## Download cbs_proj
cbs_pt = pd.read_csv('../data/projections/2021/cbs_hitter_proj.csv')

## merge playtime from proj. w/ elig
elig = elig.merge(cbs_pt['Player','BPA'], left_on='Player', right_on='Player', how='left')
elig = elig[elig['PA'] > 0]
dups = elig['Name'].duplicated(keep=False)
## view duplicated names with more than 0 PA
elig[['Name','Team']][dups].sort_values('Name')

Index(['Avail', 'Player', 'Eligible', 'BPA', 'AB', '2B', '3B', 'HR', 'BB', 'K',
       'R', 'RBI', 'SB', 'AVG', 'OBP', 'SLG', 'OPS', 'Rank', 'Name', 'Team'],
      dtype='object')

In [124]:
## left merge elig with id_map
## You might have to manaully do a few.

elig.merge(id_map[['fg_id','cbs_name', 'Avail']], left_on='Name', right_on='cbs_name')

In [83]:
## This sucks and I don't have time for it. Remember when CBS used to publish their player ids?

elig = pd.read_csv('../data/projections/2021/fg_defense.csv')[['Name', 'Team', 'Pos', 'G', 'playerid']]
def true_if_played_at(player, pos, min=10):
    '''Return true if the player played min games at pos; else false.
    '''
    if player['Pos'] == pos:
        if player['G'] >= min:
            return True
        else:
            return False
    else:
        return False

for pos in elig.Pos.unique():
    elig[pos] = elig.apply(true_if_played_at, args=(pos, 10), axis=1)



In [84]:
elig[elig['playerid'] == 1159]

,Name,Team,Pos,G,playerid,1B,3B,P,SS,C,LF,RF,2B,CF
4,Andrew Romine,TEX,SS,1,1159,False,False,False,False,False,False,False,False,False


In [85]:
elig.groupby('playerid').agg(max)

,Name,Team,Pos,G,1B,3B,P,SS,C,LF,RF,2B,CF
playerid,,,,,,,,,,,,,
785,Todd Frazier,TEX,P,29,True,True,False,False,False,False,False,False,False
1157,Tommy Hunter,PHI,P,24,False,False,True,False,False,False,False,False,False
1159,Andrew Romine,TEX,SS,1,False,False,False,False,False,False,False,False,False
1177,Albert Pujols,LAA,1B,26,True,False,False,False,False,False,False,False,False
1247,Adam Ottavino,NYY,P,24,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27458,Kwang-hyun Kim,STL,P,8,False,False,False,False,False,False,False,False,False
27459,Yoshitomo Tsutsugo,TBR,LF,16,False,True,False,False,False,True,False,False,False
27460,Shun Yamaguchi,TOR,P,17,False,False,True,False,False,False,False,False,False


In [73]:
class FantasyValuator:
    '''A class for providing Standings Points Gained based valuations of hitters.

    Attributes
    ----------
    spg: Pandas Series
        A Pandas series of SPG values.

    league_BA: float
        The expected batting average the fantasy league in which the player's value will be evaluated.         Used for caluculating xH.

    league_AB: numeric
        The number of at bats of the fantasy team the player will play for.

    league_ERA: float
        The expected ERA of the fantasy league in which the player's value will be evaluated

    ###Docs a work in progress.
    '''
    def __init__():
        self.spg = pd.Series()
        self.spg['xH'] = 8.475##used 8.47, nearly matched the real value of 8.486 in 2019.
        self.spg['HR'] = 5.286##real value was 4.782
        self.spg['R'] = 11.671###used 10.81,real value was 12.532
        self.spg['RBI'] = 13.38##used 12.20,was 14.564
        self.spg['SB'] = 3.46##used 4.27,was 2.657

        self.spg['xER'] = 8.47# was in 2018, 7.85
        self.spg['SO'] = 17.47
        self.spg['W'] = 1.73
        self.spg['SV'] = 4.55
        self.spgx['WHIP'] = 12.420
        ###was 10.60 in 2018
        ##Shit, used 10.848 in 2019; actual value in 2019 was 13.992!
        
        
        self.league_BA = .265
        self.league_ERA = 3.79
        self.league_WHIP = 1.23
        self.hitter_replacement_level = 16.72
        self.pitcher_replacement_level = 11.44
        self.catcher_replacement_level = 9##This is an approximation.

        self.hitter_categories = ['HR','xH','RBI','R','SB']
        self.pitcher_categories = ['W','SV','SO','xWHIP','xER']

    def evaluate_hitters(self, hitters, use_replacement_level=True):
        '''
        Return hitter SPG based fantasy value from fantasy stats. 
        
        Paramters
        ---------
        hitters: dict-like or dataframe
            A dataframe or dictionary with columns or indices corresponding to the hitter stats to be
            evaluated in the fantasy wins-above-replacement calculation.

        Returns
        -------
            A pd.Series indexed to hitters (for DataFrame of hitters) or a float (for a dict-like of               hitters)
        '''
        eval_data = hitters[self.hitter_categories]
        fWAR = (eval_data/spg[self.hitter_categories]).sum()
        if use_replacement_level:
            fWAR = fWAR.mask('C' in hitters['elig'], other = fWAR - self.catcher_replacement_level)
            fWAR = fWAR.where('C' in hitters['elig'], other = fWAR - self.hitter_replacement_level)
        return fWAR

    def ERA_to_xER(self,pitchers):
        '''Gets a pitchers xER.'''
        return (pitchers['ERA']-self.league_ERA)*pitchers['IP']

    def BA_to_xH(self, hitters):
        '''Get hitters xH'''
        return (hitters['BA']-self.league_BA)*hitters['AB']

    def WHIP_to_xWHIP(self, pitchers):
        return (pitchers['WHIP']-self.league_WHIP)*pitchers['IP']

    def _calculate_fWAR(self, hitter_series):

        
        
        

SyntaxError: unexpected EOF while parsing (<ipython-input-73-5e7c361de6f2>, line 83)